In [1]:
import face_recognition
from sklearn import svm
import os
import cv2
import numpy as np
from PIL import Image
from joblib import dump, load
import pandas as pd
import face_recognition_api as fra # import of created lib with neccessary functions
import time 

In [ ]:
path = "./Test" # folder for test samples
start = time.time() # time start for checking evaluation time
clf = load("./dlib_svm_face_recognition.joblib") # loading recognition model
metrics = []
folderPaths = [os.path.join(path,f) for f in os.listdir(path)] # for every person folder
for folderPath in folderPaths:
    name = folderPath.split('/')[-1] # get the name of the person
    path = folderPath
    if name!='unknown':
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)] # for every video in person folder
        for imagePath in imagePaths:
            if imagePath[-3:]=='mp4': # for every video with mp4 format
                id = imagePath[-5]
                print(name, id)
                result_path = path+'/'+id+'.txt'
                faces = fra.show_faces(imagePath, clf) # evaluate saving and showing result of face detection and checking
                results = fra.result(result_path) # get data from txt descriptors
                TP, FP = fra.check(results,faces) # compare txt descriptors and the result of build function
                TP, FN = fra.check(faces,results)
                metrics.append([TP,FP,FN])
                print(TP, FP, FN)

res_df = pd.DataFrame(metrics).rename(columns={0: "TP", 1: "FP", 2: "FN"})
res_df['precision'] = res_df.TP/(res_df.TP+res_df.FP) # calculate precision
res_df['recall'] = res_df.TP/(res_df.TP+res_df.FN) # calculate recall

print(f'Precision: {np.mean(res_df.precision):.3f} \nRecall: {np.mean(res_df.recall):.3f}')
end = time.time() - start # time end for checking evaluation time
print(end)
res_df.to_csv("./metrics/dlib_svm_results.txt", header=None, index=None, sep=' ', mode='a')

In [ ]:
# comments are the same as previous 
path = "./Test"
start = time.time()
clf = load("./dlib_knn_face_recognition.joblib") 
metrics = []
folderPaths = [os.path.join(path,f) for f in os.listdir(path)] 
for folderPath in folderPaths:
    name = folderPath.split('/')[-1]
    path = folderPath
    if name!='unknown':
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)] 
        for imagePath in imagePaths:
            if imagePath[-3:]=='mp4':
                id = imagePath[-5]
                print(name, id)
                result_path = path+'/'+id+'.txt'
                faces = fra.show_faces(imagePath, clf)
                results = fra.result(result_path)
                TP, FP = fra.check(results,faces)
                TP, FN = fra.check(faces,results)
                metrics.append([TP,FP,FN])
                print(TP, FP, FN)

res_df = pd.DataFrame(metrics).rename(columns={0: "TP", 1: "FP", 2: "FN"})
res_df['precision'] = res_df.TP/(res_df.TP+res_df.FP)
res_df['recall'] = res_df.TP/(res_df.TP+res_df.FN)

print(f'Precision: {np.mean(res_df.precision):.3f} \nRecall: {np.mean(res_df.recall):.3f}')
end = time.time() - start
print(end)
res_df.to_csv("./metrics/dlib_knn_results.txt", header=None, index=None, sep=' ', mode='a')

In [2]:
# creating the dictionary for names
start = time.time()
train_path = "./Training"
folderPaths = [os.path.join(train_path,f) for f in os.listdir(train_path)] 
names = []
ids = []
id = 0
for folderPath in folderPaths:
    name = folderPath.split('/')[-1]
    ids.append(id)
    names.append(name)
    id = id + 1
faces_dict = dict(zip(ids, names))


In [ ]:
# comments are the same as previous 
path = "./Test"
metrics = []
folderPaths = [os.path.join(path,f) for f in os.listdir(path)] 
for folderPath in folderPaths:
    name = folderPath.split('/')[-1]
    path = folderPath
    if name!='unknown':
        imagePaths = [os.path.join(path,f) for f in os.listdir(path)] 
        for imagePath in imagePaths:
            if imagePath[-3:]=='mp4':
                id = imagePath[-5]
                print(name, id)
                result_path = path+'/'+id+'.txt'
                faces = fra.show_faces_cv2(imagePath,faces_dict)
                results = fra.result(result_path)
                TP, FP = fra.check(results,faces)
                TP, FN = fra.check(faces,results)
                metrics.append([TP,FP,FN])
                print(TP, FP, FN)

res_df = pd.DataFrame(metrics).rename(columns={0: "TP", 1: "FP", 2: "FN"})
res_df['precision'] = res_df.TP/(res_df.TP+res_df.FP)
res_df['recall'] = res_df.TP/(res_df.TP+res_df.FN)

print(f'Precision: {np.mean(res_df.precision):.3f} \nRecall: {np.mean(res_df.recall):.3f}')
end = time.time() - start
print(end)
res_df.to_csv("./metrics/cv2.txt", header=None, index=None, sep=' ', mode='a')

In [13]:
# delete all opened windows
cv2.destroyAllWindows()